# Spherical Harmonics

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.special import sph_harm
from matplotlib import animation
from matplotlib.colors import LightSource
from IPython import display
import matplotlib
import os

In [ ]:
matplotlib.use('TkAgg')

In [ ]:
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2*np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

# The Cartesian coordinates of the unit sphere
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

def get_SphHarm(l, m, component):
    ## Calculate the spherical harmonic Y(l,m) and normalize to [0,1]
    if component=='abs':
        rx = getattr(sph_harm(m, l, phi, theta), 'real')
        ix = getattr(sph_harm(m, l, phi, theta), 'imag')
        fcolors = np.sqrt(rx*rx+ix*ix)
    else:
        fcolors = getattr(sph_harm(m, l, phi, theta), component)
    wavefun = fcolors
    fmax, fmin = fcolors.max(), fcolors.min()
    # scales [max, min] so 0.5 is always 0 and max or min is on the edge
    absmax = max(abs(fmax),abs(fmin))
    if absmax == 0:
        fcolors += 0.5
    else:
        fcolors = fcolors/(2*absmax) + 0.5

    # scales [min,max] to [0,1]
    #if (fmax==fmin):
    #    fcolors = (fcolors - fmin)
    #else:
    #    fcolors = (fcolors - fmin)/(fmax - fmin)
    ls = LightSource(180, 45)
    rgb = ls.shade(fcolors, cmap=cm.seismic, vert_exag=0.1, blend_mode='soft', norm=matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True))
    # if np.unique(wavefun).size==1:
    #     rgb = np.tile(np.array([1,0,0,1]), (rgb.shape[0],rgb.shape[1],1))
    return wavefun,rgb
    
def generate_animation(rgb, r, fig, ax):
    ## Generate animation
    def init():
        for rgbi,axi,ri in zip(rgb,ax,r):
            ri = np.abs(np.squeeze(np.array(ri)))
            axi.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=rgbi, shade=False)
        return fig,
    def animate(i):
        for axi in ax:
            axi.view_init(elev=30, azim=0+10*i)
        return fig,
    
    ani = animation.FuncAnimation(fig, animate, init_func=init,
                           frames=36, interval=50, blit=True)
    return ani

def generate_animation2(rgb, r, fig, ax):
    ## Generate animation
    def init():
        for rgbi,axi,ri in zip(rgb,ax,r):
            ri = np.abs(np.squeeze(np.array(ri)))
            axi.plot_surface(x*ri, y*ri, z*ri,  rstride=1, cstride=1, facecolors=rgbi, shade=False)
        return fig,
    def animate(i):
        for axi in ax:
            axi.view_init(elev=30, azim=0+10*i)
        return fig,
    ani = animation.FuncAnimation(fig, animate, init_func=init,
                           frames=36, interval=50, blit=True)
    return ani

def createAllHarmonics(l):
    path = "./testHiDPI/"
    try:
        os.mkdir(path)
    except:
        pass
    plt.rcParams['figure.facecolor'] = '#d3d3d3'
    for m in range(-l,l+1):
        print(l,m)
        fig = plt.figure(figsize=plt.figaspect(0.5))

        realR,realPart = get_SphHarm(l, m, 'real')
        imagR,imagPart = get_SphHarm(l, m, 'imag')
        absR,absValue = get_SphHarm(l, m, 'abs')
        absR2 = absR**2
        #print("rR\n",realR,"\nrP\n",realPart,"\niR\n",imagR,"\niP\n",imagPart)


        realAx = fig.add_subplot(131, projection='3d')
        imagAx = fig.add_subplot(132, projection='3d')
        absAx = fig.add_subplot(133, projection='3d')
        allaxs = [realAx, imagAx,absAx]

        for eachAx in allaxs:
            eachAx.set_axis_off()
            eachAx.set_facecolor(color='#d3d3d3')

        aniflat = generate_animation([realPart,imagPart,absValue],[realR,imagR,absR2],fig,allaxs)
        anidist = generate_animation2([realPart,imagPart,absValue],[realR,imagR,absR2],fig,allaxs)
        #aniabs = generate_animation2([absValue,absValue],[absR,absR2],fig,allaxs)
        ## Display animation
        print('Drawing ...')

        
        FFwriter = animation.FFMpegWriter(fps=30, codec="libx264")     
        anidist.save(path+f"l{l}m{m}dist.mp4", writer = FFwriter, dpi=300)
        aniflat.save(path+f"l{l}m{m}flat.mp4", writer = FFwriter, dpi=300)

        #video = ani.to_html5_video()
        #html = display.HTML(video)
        #display.HTML(video)
        #display.display(html)
        plt.close(fig)
    print("done")

In [ ]:
for l in range(1,6):
    createAllHarmonics(l)